In [1]:
from collections import defaultdict
import random
import numpy as np
np.set_printoptions(precision=2, suppress=True)

import time
import copy 
import multiprocess as mp

import gym
from env import FrozenLakeCustom, FrozenLakeSimulator

from mcts_haver import run_mcts_trial
from value_iteration import value_iteration

from config import parse_args
from utils import MultiProcess

import logging
logger = logging.getLogger()
logger.setLevel(logging.FATAL)

In [2]:
np.random.seed(0)
random.seed(0)

# params
args = parse_args()

#
env_id = "FrozenLake-v1"
env = FrozenLakeCustom(
    map_name=args["map_name"], is_slippery=args["is_slippery"],
    render_mode=args["render_mode"])

simulator = FrozenLakeSimulator(env.P)

V_vit, Q_vit = value_iteration(
    simulator, args["gamma"], args["vit_thres"])
# global Q_vit_g = Q_vit
        
for state in range(simulator.num_states):
    logging.warning(f"\n-> state = {state}")
    logging.warning(f"V[state] = {V_vit[state]:0.4f}")
    for action in range(simulator.num_actions):
        logging.warning(f"Q[state][action] = {Q_vit[state][action]:0.4f}")
    logging.warning(f"best_action={np.argmax(Q_vit[state])}")
    
manager = mp.Manager()
ep_reward_list = manager.list()
Q_mcts_list = manager.list()

def run_trial(i_trial, Q_vit, args):

    random.seed(10000+i_trial)
    np.random.seed(10000+i_trial)

    env = FrozenLakeCustom(
        map_name=args["map_name"], is_slippery=args["is_slippery"],
        render_mode=args["render_mode"])

    simulator = FrozenLakeSimulator(env.P)

    Q_mcts, ep_reward = run_mcts_trial(env, simulator, Q_vit, i_trial, args)

    ep_reward_list.append(ep_reward)
    Q_mcts_list.append(Q_mcts)
    return ep_reward

In [4]:
args["update_method"] = "haver"
args["rollout_method"] = ""

print(f"num_trials = {args['num_trials']}")
# print(f"mcts_num_trajectories = {args['mcts_num_trajectories']}")


hparam_ucb_scale_list = np.arange(10, 100, 10)
hparam_ucb_scale_list = [1, 2, 4, 8, 16, 32, 64, 128]
# hparam_ucb_scale_list = [2**i for i in range(1, 9)]
# args["hparam_ucb_scale"] = 64

hparam_haver_std_list = np.arange(10, 100, 10)
hparam_haver_std_list = [1, 2, 4, 8, 16, 32, 64, 128]
# hparam_haver_std_list = [2**i for i in range(1, 9)]
args["hparam_haver_var"] = 100**2


num_trajectories_list = [200, 500, 1000, 1500, 2000, 2500, 3000]
# num_trajectories_list = [200, 500, 800]
# num_trajectories_list = [2]
best_param_list = []
max_reward_mean_list = []
res_text1 = ""
res_text2 = ""
for num_trajectories in num_trajectories_list:
    print(f"\n-> num_trajectories = {num_trajectories}")
    args["mcts_num_trajectories"] = num_trajectories
    
    best_param = None
    max_reward_mean = -np.inf
    start_time = time.time()
    res_text1 += f"{num_trajectories} "
    res_text2 += f"{num_trajectories} "
    for hparam_ucb_scale in hparam_ucb_scale_list:
        # start_time = time.time()
        
        args["hparam_ucb_scale"] = hparam_ucb_scale
        print(f"hparam_ucb_scale = {args['hparam_ucb_scale']}")
        # print(f"hparam_haver_var = {args['hparam_haver_var']}")
        
        pool = mp.Pool()
        pool.starmap(run_trial, [(i, Q_vit, args) for i in range(args["num_trials"])])

        reward_mean = np.mean(ep_reward_list)
        reward_std = np.std(ep_reward_list, ddof=1) if len(ep_reward_list) > 1 else 0
        reward_error = reward_std/np.sqrt(args["num_trials"])
        if hparam_ucb_scale <= 8:
            res_text1 += f"& {reward_mean:0.2f} (\u00B1{reward_error:0.2f}) "
        else:
            res_text2 += f"& {reward_mean:0.2f} (\u00B1{reward_error:0.2f}) "
        print(f"reward = {reward_mean:0.2f} +/- {reward_error:0.2f}")

        # if reward_mean > max_reward_mean:
        #     max_reward_mean = reward_mean 
        #     best_param = hparam_haver_std
    
        ep_reward_list[:] = []
        Q_mcts_list[:] = []
    
        end_time = time.time()
        # print(f"it takes {end_time-start_time:0.4f}")
    
    res_text1 += "\\\\ \n \hline \n"
    res_text2 += "\\\\ \n \hline \n"
    
    # print(f"max_reward_mean = {max_reward_mean:0.2f}")
    print(f"it takes {end_time-start_time:0.4f}")
    
    # max_reward_mean_list.append(max_reward_mean)
    # best_param_list.append(best_param)

num_trials = 20

-> num_trajectories = 200
hparam_ucb_scale = 1
reward = -101.05 +/- 0.05
hparam_ucb_scale = 2
reward = -101.05 +/- 0.05
hparam_ucb_scale = 4
reward = -101.15 +/- 0.08
hparam_ucb_scale = 8
reward = -81.80 +/- 8.93
hparam_ucb_scale = 16
reward = -52.75 +/- 11.12
hparam_ucb_scale = 32
reward = -52.50 +/- 11.13
hparam_ucb_scale = 64
reward = -86.45 +/- 7.95
hparam_ucb_scale = 128
reward = -81.60 +/- 8.90
it takes 8.8611

-> num_trajectories = 500
hparam_ucb_scale = 1
reward = -101.05 +/- 0.05
hparam_ucb_scale = 2
reward = -101.10 +/- 0.07
hparam_ucb_scale = 4
reward = -91.55 +/- 6.62
hparam_ucb_scale = 8
reward = -77.00 +/- 9.59
hparam_ucb_scale = 16
reward = -52.90 +/- 11.04
hparam_ucb_scale = 32
reward = -28.30 +/- 9.63
hparam_ucb_scale = 64
reward = -52.50 +/- 11.13
hparam_ucb_scale = 128
reward = -67.05 +/- 10.61
it takes 23.5498

-> num_trajectories = 1000
hparam_ucb_scale = 1
reward = -101.05 +/- 0.05
hparam_ucb_scale = 2
reward = -91.55 +/- 6.62
hparam_ucb_scale = 4

Process ForkPoolWorker-144:
Process ForkPoolWorker-161:
Process ForkPoolWorker-55:
Process ForkPoolWorker-132:
Process ForkPoolWorker-153:
Process ForkPoolWorker-212:
Process ForkPoolWorker-147:
Process ForkPoolWorker-134:
Process ForkPoolWorker-108:
Process ForkPoolWorker-97:
Process ForkPoolWorker-95:
Process ForkPoolWorker-50:
Process ForkPoolWorker-86:
Process ForkPoolWorker-47:
Process ForkPoolWorker-69:
Process ForkPoolWorker-49:
Process ForkPoolWorker-445:
Process ForkPoolWorker-107:
Traceback (most recent call last):
Process ForkPoolWorker-427:
Process ForkPoolWorker-386:
Process ForkPoolWorker-59:
Process ForkPoolWorker-133:
Process ForkPoolWorker-139:
Process ForkPoolWorker-114:
Process ForkPoolWorker-92:
Traceback (most recent call last):
Process ForkPoolWorker-71:
Process ForkPoolWorker-96:
Process ForkPoolWorker-336:
Traceback (most recent call last):
Traceback (most recent call last):
Process ForkPoolWorker-75:
Process ForkPoolWorker-26:
Traceback (most recent call last):

KeyboardInterrupt: 

  File "/home/tnn/miniconda3/envs/tnn1/lib/python3.7/site-packages/multiprocess/process.py", line 297, in _bootstrap
    self.run()
Traceback (most recent call last):
Process ForkPoolWorker-62:
Traceback (most recent call last):
Process ForkPoolWorker-70:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/tnn/miniconda3/envs/tnn1/lib/python3.7/site-packages/multiprocess/process.py", line 297, in _bootstrap
    self.run()
Traceback (most recent call last):
Process ForkPoolWorker-429:
Process ForkPoolWorker-87:
Process ForkPoolWorker-64:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/tnn/miniconda3/envs/tnn1/lib/python3.7/site-packages/multiprocess/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
Traceback (most recent call last):
Process ForkPoolWorker-142:
  File "/home/tnn/miniconda3/envs/tnn1/lib/python3.7/site-packages/

In [ ]:
print(res_text1)
print(res_text2)